# Perfilamiento de trafico de test

## Resumen

En el siguiente informe se resaltan las actividades prepuestas en la reunión del dia 23/11/2108 [The effects of DoS attacks on ODL and POX SDN controllers](https://ieeexplore.ieee.org/document/8080058) y las conclusiones al respecto.

## Actividades realizadas 

### 1. Codificación del banco de pruebas en mininet 

En el archivo [test_experiment_beta1.py](./test_experiment_beta1.py) automatiza (en teoria los resultados). Por comodidad se muestra este codigo a continuación:

```python
import sys

from mininet.topo import Topo
from mininet.net import Mininet
from mininet.util import irange,dumpNodeConnections
from mininet.log import setLogLevel, info
from mininet.link import TCLink

from time import time
from select import poll, POLLIN
from subprocess import Popen, PIPE
from time import sleep


class TopoTest(Topo):
   "Linear topology of k switches, with one host per switch."

   def __init__(self, nSwitch = 2, nHosts = 4, **opts):
       """Init.
           nSwitch: number of switches
           nHosts: numero de hosts
           hconf: host configuration options
           lconf: link configuration options
        """

       super(TopoTest, self).__init__(**opts)

       self.nSwitch = nSwitch
       self.nHosts = nHosts
       h = 0 #nHosts - 1
       lastSwitch = None
       for i in irange(1, nSwitch):
           switch = self.addSwitch('s%s' % i)
           for j in irange(h + 1, h + nHosts):
               # bw = 10 Mbps
               host = self.addHost('h%s' % j)
               self.addLink(switch, host, bw = 10)
           if lastSwitch:
               # bw = 10 Mbps
               self.addLink(switch, lastSwitch, bw = 1000)
           lastSwitch = switch
           h += nHosts

def test_ping():
    topo = TopoTest()
    net = Mininet(topo=topo, link=TCLink)
    net.start()
    net.pingAll()
    net.stop()

def testBW(test_time, experimento):
    topo = TopoTest()
    net = Mininet(topo=topo, link=TCLink)
    net.start()
    hosts_iperf = [net.get(experimento['iperf_src']), net.get(experimento['iperf_dst'])]
    outfile = "iperf_" + \
              experimento['iperf_src'] + \
              "-" + \
              experimento['iperf_dst'] + \
              ".log"

    hosts_iperf[1].cmd('echo >', outfile)
    # Iperf server (Se redirecciona solo la salida)
    hosts_iperf[1].cmdPrint('iperf', '-s', '-i', '1',
                            '>', outfile,
                            '&')
    hosts_iperf[0].cmdPrint('iperf', '-c', str(hosts_iperf[1].IP()), '-t ' + str(test_time) + " &")
    sleep(test_time)
    net.pingAll()
    net.stop()


def testBW_ataque(test_time, experimento):
    topo = TopoTest()
    net = Mininet(topo=topo, link=TCLink)
    net.start()
    hosts_iperf = [net.get(experimento['iperf_src']), net.get(experimento['iperf_dst'])]
    a_list = experimento['atacantes'].split("-")
    atacantes = []
    victima = net.get(experimento['victima'])
    for a in a_list:
        atacantes.append(net.get(a))

    # Archivo de salida para el iperf
    outfile = "iperf_" + \
              experimento['iperf_src'] + \
              "-" + \
              experimento['iperf_dst'] + \
              ".log"

    # Redireccion para vaciar el archivo

    hosts_iperf[1].cmd('echo >', outfile)


    # Iperf server (Se redirecciona solo la salida)
    hosts_iperf[1].cmdPrint('iperf', '-s', '-i', '1',
                            '>', outfile,
                            '&')

    # Iperf_client
    hosts_iperf[0].cmdPrint('iperf', '-c', str(hosts_iperf[1].IP()), '-t ' + str(test_time) + " &")

    # Lanzando el ataque
    info("*** Lanzado ataques...")
    for a in atacantes:
        # a.cmdPrint('hping3 --flood --rand-source', str(victima.IP()), " &")
        # a.cmdPrint('ping -c 10', str(victima.IP()), " &")
        # a.cmdPrint('hping3 --flood', str(victima.IP()), " &")
        a.cmdPrint('hping3 --rand-source', str(victima.IP()), "&")

    sleep(test_time)
    net.stop()


if __name__ == '__main__':
    print ("Empezando el ensayo")
    setLogLevel('info')
    experimento1 = {'iperf_src': 'h1','iperf_dst': 'h8'}
    experimento2 = {'iperf_src': 'h1','iperf_dst': 'h8', 'victima':'h5' , 'atacantes':'h2'}
    # test_ping()
    #testBW(50, experimento1)
    testBW_ataque(20,experimento2)

    print ("Hasta la vista baby")
```

### 2. Algunos de analisis previamente realizados

A Continuación se exponen algunos de los resultados obtenidos:

1. **Resultados sin ataque**: Se llama la funcion ```testBW``` mediante el siguiente experimento:

```python
...
experimento1 = {'iperf_src': 'h1','iperf_dst': 'h8'}
testBW(50, experimento1)
...
```

El resultado sin el ataque es un archivo de [texto](iperf_h1-h8_no_attack.log) con un contenido como el siguiente:

```
------------------------------------------------------------
Server listening on TCP port 5001
TCP window size: 85.3 KByte (default)
------------------------------------------------------------
[  4] local 10.0.0.8 port 5001 connected with 10.0.0.1 port 59002
[ ID] Interval       Transfer     Bandwidth
[  4]  0.0- 1.0 sec  1.14 MBytes  9.57 Mbits/sec
[  4]  1.0- 2.0 sec  1.14 MBytes  9.55 Mbits/sec
[  4]  2.0- 3.0 sec  1.14 MBytes  9.57 Mbits/sec
[  4]  3.0- 4.0 sec  1.14 MBytes  9.56 Mbits/sec
[  4]  4.0- 5.0 sec  1.14 MBytes  9.55 Mbits/sec
[  4]  5.0- 6.0 sec  1.14 MBytes  9.57 Mbits/sec
...
[  4] 47.0-48.0 sec  1.14 MBytes  9.59 Mbits/sec
[  4] 48.0-49.0 sec  1.14 MBytes  9.57 Mbits/sec
[  4] 49.0-50.0 sec  1.14 MBytes  9.55 Mbits/sec
[  4]  0.0-50.3 sec  57.4 MBytes  9.56 Mbits/sec
```

2. **Resultados empleando un ping**: Se llama la funcion ```testBW_ataque``` mediante el siguiente experimento:

```python
...
experimento2 = {'iperf_src': 'h1','iperf_dst': 'h8', 'victima':'h5' , 'atacantes':'h2'}
testBW_ataque(50, experimento2)
...
```

Para el caso, en la función, se empleo un comando ping en lo que llamamos el atacante, asi:

```python
def testBW_ataque(test_time, experimento):
    ...
    a.cmdPrint('ping -c 4', str(victima.IP()), " &")
    ...
```

El resultado sin el ataque es un archivo de [texto](iperf_h1-h8_ping_h2-h5.log) con un contenido como el siguiente:

```
------------------------------------------------------------
Server listening on TCP port 5001
TCP window size: 85.3 KByte (default)
------------------------------------------------------------
[  4] local 10.0.0.8 port 5001 connected with 10.0.0.1 port 59156
[ ID] Interval       Transfer     Bandwidth
[  4]  0.0- 1.0 sec  1.14 MBytes  9.58 Mbits/sec
[  4]  1.0- 2.0 sec  1.14 MBytes  9.53 Mbits/sec
[  4]  2.0- 3.0 sec  1.14 MBytes  9.56 Mbits/sec
[  4]  3.0- 4.0 sec  1.14 MBytes  9.57 Mbits/sec
[  4]  4.0- 5.0 sec  1.14 MBytes  9.55 Mbits/sec
[  4]  5.0- 6.0 sec  1.14 MBytes  9.56 Mbits/sec
[  4]  6.0- 7.0 sec  1.14 MBytes  9.57 Mbits/sec
...
[  4] 45.0-46.0 sec  1.14 MBytes  9.57 Mbits/sec
[  4] 46.0-47.0 sec  1.14 MBytes  9.56 Mbits/sec
[  4] 47.0-48.0 sec  1.14 MBytes  9.57 Mbits/sec
[  4] 48.0-49.0 sec  1.14 MBytes  9.58 Mbits/sec
[  4] 49.0-50.0 sec  1.14 MBytes  9.52 Mbits/sec
```

Tengase en cuenta, que si se aumentan el numero de pings, es posible que la salida se afecte, por lo menos, vemos que ya de por si, en cuanto a la ultima fila ya hay una varianza (Es posible que para escribir datos en el archivo de salida se necesite mas tiempo).

3. **Resultados empleando un spoofing**: Se llama la funcion ```testBW_ataque``` mediante el siguiente experimento en el cual la idea es falsificar la IP origen:

```python
experimento2 = {'iperf_src': 'h1','iperf_dst': 'h8', 'victima':'h5' , 'atacantes':'h2'}
testBW_ataque(50, experimento2)
```

Para el caso, en la función, se empleo un comando ping en lo que llamamos el atacante, asi:

```python
def testBW_ataque(test_time, experimento):
    ...
    a.cmdPrint('hping3 --rand-source', str(victima.IP()), "&")
    ...
```

El resultado sin el ataque es un archivo de [texto](iperf_h1-h8_spoofing.log) con un contenido como el siguiente:

```
------------------------------------------------------------
Server listening on TCP port 5001
TCP window size: 85.3 KByte (default)
------------------------------------------------------------
[  4] local 10.0.0.8 port 5001 connected with 10.0.0.1 port 59238
[ ID] Interval       Transfer     Bandwidth
[  4]  0.0- 1.0 sec  1.14 MBytes  9.60 Mbits/sec
[  4]  1.0- 2.0 sec  1.14 MBytes  9.55 Mbits/sec
[  4]  2.0- 3.0 sec  1.14 MBytes  9.56 Mbits/sec
[  4]  3.0- 4.0 sec  1.14 MBytes  9.57 Mbits/sec
[  4]  4.0- 5.0 sec  1.14 MBytes  9.55 Mbits/sec
[  4]  5.0- 6.0 sec  1.14 MBytes  9.56 Mbits/sec
[  4]  6.0- 7.0 sec  1.14 MBytes  9.57 Mbits/sec
...
[  4] 46.0-47.0 sec  1.14 MBytes  9.57 Mbits/sec
[  4] 47.0-48.0 sec  1.14 MBytes  9.57 Mbits/sec
[  4] 48.0-49.0 sec  1.14 MBytes  9.56 Mbits/sec
[  4] 49.0-50.0 sec  1.14 MBytes  9.57 Mbits/sec
```

3. **Resultados empleando flooding**: Se llama la funcion ```testBW_ataque``` mediante el siguiente experimento generando muchos paquetes para inundar la red:

```python
experimento2 = {'iperf_src': 'h1','iperf_dst': 'h8', 'victima':'h5' , 'atacantes':'h2'}
testBW_ataque(50, experimento2)
```

Para el caso, en la función, se empleo un comando ping en lo que llamamos el atacante, asi:

```python
def testBW_ataque(test_time, experimento):
    ...
    a.cmdPrint('hping3 --flood', str(victima.IP()), " &")    
    ...
```

El resultado sin el ataque es un archivo de [texto](iperf_h1-h8_flooding.log) con un contenido como el siguiente:

```
------------------------------------------------------------
Server listening on TCP port 5001
TCP window size: 85.3 KByte (default)
------------------------------------------------------------
[  4] local 10.0.0.8 port 5001 connected with 10.0.0.1 port 59306
[ ID] Interval       Transfer     Bandwidth
[  4]  0.0- 1.0 sec  1.14 MBytes  9.58 Mbits/sec
[  4]  1.0- 2.0 sec  1.14 MBytes  9.56 Mbits/sec
[  4]  2.0- 3.0 sec  1.14 MBytes  9.55 Mbits/sec
[  4]  3.0- 4.0 sec  1.14 MBytes  9.55 Mbits/sec
[  4]  4.0- 5.0 sec  1.14 MBytes  9.57 Mbits/sec
[  4]  5.0- 6.0 sec  1.14 MBytes  9.56 Mbits/sec
[  4]  6.0- 7.0 sec  1.14 MBytes  9.57 Mbits/sec
[  4]  7.0- 8.0 sec  1.14 MBytes  9.57 Mbits/sec

```

El archivo anterior no tiene mas filas, para el caso dado es como si el ancho de banda se ubiera ido a 0 pues se presume que dejaron de llegar paquetes al servidor. No se que tan relevante sea esto o que sentido tienen estos resultados con la realidad. Si lo anterior sucede, una cosa que si me extraña es la caida tan abrupta del ancho de banda. No se si esto tenga que ver con el intervalo fijado en la medicion pues puede que un segundo sea mucho.

3. **Resultados empleando flooding y spoofing**: Se llama la funcion ```testBW_ataque``` mediante el siguiente experimento generando muchos paquetes para inundar la red:

```python
experimento2 = {'iperf_src': 'h1','iperf_dst': 'h8', 'victima':'h5' , 'atacantes':'h2'}
testBW_ataque(50, experimento2)
```

Para el caso, en la función, se empleo un comando ping en lo que llamamos el atacante, asi:

```python
def testBW_ataque(test_time, experimento):
    ...
    a.cmdPrint('hping3 --flood --rand-source', str(victima.IP()), " &")    
    ...
```

El resultado sin el ataque es un archivo de [texto](iperf_h1-h8_spoof_flood.log) con un contenido como el siguiente:

```
------------------------------------------------------------
Server listening on TCP port 5001
TCP window size: 85.3 KByte (default)
------------------------------------------------------------
[  4] local 10.0.0.8 port 5001 connected with 10.0.0.1 port 59844
[ ID] Interval       Transfer     Bandwidth
[  4]  0.0- 1.0 sec  1.14 MBytes  9.57 Mbits/sec
[  4]  1.0- 2.0 sec  1.14 MBytes  9.56 Mbits/sec
[  4]  2.0- 3.0 sec  1.14 MBytes  9.56 Mbits/sec
[  4]  3.0- 4.0 sec  1.14 MBytes  9.55 Mbits/sec
[  4]  4.0- 5.0 sec  1.14 MBytes  9.57 Mbits/sec
[  4]  5.0- 6.0 sec  1.14 MBytes  9.56 Mbits/sec
[  4]  6.0- 7.0 sec  1.14 MBytes  9.57 Mbits/sec
[  4]  7.0- 8.0 sec  1.14 MBytes  9.57 Mbits/sec
```

Solo resta decir que la salida es muy similar al caso de flooding

## Conclusiones 

1. Oh! y ahora quien podra ayudarme.